In [52]:
import pandas as pd
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker
import logging
from datetime import datetime
import uuid

# Конфигурация базы данных
DB_CONFIG = {
    'host': 'localhost',  # или IP-адрес Docker контейнера
    'port': '5432',
    'database': 'server_metrics',  # имя вашей базы данных
    'user': 'postgres',
    'password': 'postgres'  # замените на ваш пароль
}

# Настройка логирования
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

def create_database_connection():
    """Создание соединения с базой данных"""
    try:
        # Формируем строку подключения
        connection_string = f"postgresql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"

        # Создаем engine
        engine = create_engine(connection_string)

        # Проверяем соединение
        with engine.connect() as conn:
            conn.execute(text("SELECT 1"))

        logger.info("Успешное подключение к базе данных")
        return engine

    except Exception as e:
        logger.error(f"Ошибка подключения к базе данных: {e}")
        raise

def read_excel_file(file_path):
    """Чтение данных из Excel файла"""
    try:
        # Читаем Excel файл
        df = pd.read_excel(file_path, sheet_name=0)  # sheet_name=0 для первого листа

        # Проверяем наличие необходимых колонок
        required_columns = ['vm', 'date', 'metric', 'max_value', 'min_value', 'avg_value']
        missing_columns = [col for col in required_columns if col not in df.columns]

        if missing_columns:
            raise ValueError(f"Отсутствуют необходимые колонки: {missing_columns}")

        logger.info(f"Успешно прочитан файл: {file_path}")
        logger.info(f"Количество строк: {len(df)}")
        logger.info(f"Колонки: {df.columns.tolist()}")

        return df

    except Exception as e:
        logger.error(f"Ошибка чтения Excel файла: {e}")
        raise

def prepare_data(df):
    """Подготовка данных для вставки"""
    try:
        # Создаем копию данных
        data = df.copy()

        # Преобразуем дату в формат datetime
        data['date'] = pd.to_datetime(data['date'], errors='coerce')

        # Проверяем корректность преобразования дат
        invalid_dates = data['date'].isna().sum()
        if invalid_dates > 0:
            logger.warning(f"Найдено {invalid_dates} некорректных дат")

        # Преобразуем числовые колонки
        numeric_columns = ['max_value', 'min_value', 'avg_value']
        for col in numeric_columns:
            data[col] = pd.to_numeric(data[col], errors='coerce')

        # Генерируем UUID для каждой строки
        data['id'] = [str(uuid.uuid4()) for _ in range(len(data))]

        # Добавляем created_at
        data['created_at'] = datetime.now()

        # Выбираем только необходимые колонки в правильном порядке
        final_columns = ['id', 'vm', 'date', 'metric', 'max_value', 'min_value', 'avg_value', 'created_at']
        data = data[final_columns]

        # Удаляем строки с NaN значениями в ключевых полях
        original_count = len(data)
        data = data.dropna(subset=['vm', 'date', 'metric'])
        removed_count = original_count - len(data)

        if removed_count > 0:
            logger.warning(f"Удалено {removed_count} строк с отсутствующими ключевыми значениями")

        logger.info(f"Подготовлено {len(data)} строк для вставки")

        return data

    except Exception as e:
        logger.error(f"Ошибка подготовки данных: {e}")
        raise

def insert_data_to_db(engine, data):
    """Вставка данных в базу данных"""
    try:
        # Создаем сессию
        Session = sessionmaker(bind=engine)
        session = Session()

        try:
            # Сначала очистим таблицу (опционально)
            # Если хотите добавить к существующим данным, закомментируйте эту строку
            # session.execute(text("TRUNCATE TABLE server_metrics RESTART IDENTITY CASCADE"))
            # logger.info("Таблица очищена")

            # Используем pandas для массовой вставки
            data.to_sql(
                name='server_metrics',
                con=engine,
                if_exists='append',
                index=False,
                method='multi',  # Используем multi для ускорения
                chunksize=1000
            )

            session.commit()
            logger.info(f"Успешно вставлено {len(data)} строк в базу данных")

            # Проверяем количество записей в базе
            result = session.execute(text("SELECT COUNT(*) FROM server_metrics")).fetchone()
            logger.info(f"Всего записей в базе: {result[0]}")

            return True

        except Exception as e:
            session.rollback()
            logger.error(f"Ошибка при вставке данных: {e}")
            raise

        finally:
            session.close()

    except Exception as e:
        logger.error(f"Ошибка работы с базой данных: {e}")
        raise

def validate_data_in_db(engine):
    """Проверка данных в базе"""
    try:
        with engine.connect() as conn:
            # Проверяем количество уникальных VM
            vm_count = conn.execute(
                text("SELECT COUNT(DISTINCT vm) FROM server_metrics")
            ).fetchone()[0]
            logger.info(f"Уникальных VM в базе: {vm_count}")

            # Проверяем уникальные метрики
            metrics = conn.execute(
                text("SELECT DISTINCT metric FROM server_metrics ORDER BY metric")
            ).fetchall()
            logger.info(f"Уникальные метрики в базе: {[m[0] for m in metrics]}")

            # Проверяем диапазон дат
            date_range = conn.execute(
                text("SELECT MIN(date), MAX(date) FROM server_metrics")
            ).fetchone()
            logger.info(f"Диапазон дат: с {date_range[0]} по {date_range[1]}")

    except Exception as e:
        logger.error(f"Ошибка при проверке данных: {e}")

def main():
    """Основная функция"""
    excel_file_path = "../data/metrics.xlsx"  # Путь к вашему Excel файлу

    try:
        # Шаг 1: Подключение к базе данных
        engine = create_database_connection()

        # Шаг 2: Чтение данных из Excel
        df = read_excel_file(excel_file_path)

        # Шаг 3: Подготовка данных
        prepared_data = prepare_data(df)

        # Шаг 4: Вставка данных в базу
        insert_data_to_db(engine, prepared_data)

        # Шаг 5: Проверка данных
        validate_data_in_db(engine)

        logger.info("Загрузка данных завершена успешно!")

    except Exception as e:
        logger.error(f"Критическая ошибка: {e}")
        return False

    return True

if __name__ == "__main__":
    # Запуск скрипта
    success = main()

    if success:
        print("\n" + "="*50)
        print("СКРИПТ УСПЕШНО ВЫПОЛНЕН!")
        print("="*50)
        exit(0)
    else:
        print("\n" + "="*50)
        print("СКРИПТ ЗАВЕРШИЛСЯ С ОШИБКАМИ!")
        print("="*50)
        exit(1)

2025-12-15 09:38:48,231 - INFO - Успешное подключение к базе данных
2025-12-15 09:38:48,337 - INFO - Успешно прочитан файл: ../data/metrics.xlsx
2025-12-15 09:38:48,337 - INFO - Количество строк: 2240
2025-12-15 09:38:48,338 - INFO - Колонки: ['vm', 'date', 'metric', 'max_value', 'min_value', 'avg_value']
2025-12-15 09:38:48,350 - INFO - Подготовлено 2240 строк для вставки
/var/folders/n6/9hwnx40x1g53w79wdp1sqz5w0000gn/T/ipykernel_30577/1091840624.py:126: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data.to_sql(
2025-12-15 09:38:48,351 - ERROR - Ошибка при вставке данных: 'Engine' object has no attribute 'cursor'
2025-12-15 09:38:48,352 - ERROR - Ошибка работы с базой данных: 'Engine' object has no attribute 'cursor'
2025-12-15 09:38:48,352 - ERROR - Критическая ошибка: 'Engine' object has no attribute 'cursor'



СКРИПТ ЗАВЕРШИЛСЯ С ОШИБКАМИ!
